<a href="https://colab.research.google.com/github/danobaro/BigDataSecureSys/blob/main/RNNLSTM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install scikeras  # Install the scikeras package

In [3]:
!pip install tensorflow
!pip install interpret
!pip install shap
!pip install lime

In [4]:
!pip install pandas

In [5]:
!pip install --upgrade tensorflow

In [8]:
!pip install --upgrade scikit-learn

In [9]:
!pip install --upgrade scikit-learn # Upgrade scikit-learn to the latest version

In [10]:
!pip install --upgrade scikit-learn
!pip install --upgrade scikeras

In [11]:
!pip uninstall scikit-learn scikeras -y
!pip install scikit-learn scikeras

Found existing installation: scikit-learn 1.5.0
Uninstalling scikit-learn-1.5.0:
  Successfully uninstalled scikit-learn-1.5.0
Found existing installation: scikeras 0.13.0
Uninstalling scikeras-0.13.0:
  Successfully uninstalled scikeras-0.13.0
  Using cached scikit_learn-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)
  Using cached scikeras-0.13.0-py3-none-any.whl (26 kB)


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
data = pd.read_csv("/content/drive/My Drive/ffh_dataset/PID1/PID_1.csv")
data.head()
print(data.columns)

Index(['Datetime', ' ECGR ', ' AvgECG ', ' ampECG ', ' MAX ', ' GSRR ',
       ' Resistance ', ' GSR(kohm) ', ' SCL ', ' GSRRMS ', ' GSRMAV ',
       ' EMGR ', ' EMGRMS ', ' EMGMAV ', ' EMGVAR ', ' EMGSSI ', ' IEMG ',
       ' AccX ', ' AccY', ' AccZ ', ' GyroX ', ' GyroY ', ' GyroZ ', ' Aroll ',
       ' Apitch ', ' Groll ', ' Gpitch', ' Gy ', ' Combroll ', ' Combpitch',
       'Features'],
      dtype='object')


In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score

# Load data from CSV
# data = pd.read_csv("fall_data.csv")

# Define a function to map features to fall/no-fall categories
def map_features(features):
  # Note: Assuming 'features' is now an integer representing the encoded feature
  fall_activities = [0, 1, 2] # Replace with the encoded values for fall activities
  if features in fall_activities:
    return 0  # Fall
  else:
    return 1  # No Fall

# Apply mapping function to 'Features' column and create a new 'Fall' column
data["Fall"] = data["Features"].apply(map_features) # 'Features' column now contains integers

# Encode categorical columns (optional, if needed)
label_encoder = LabelEncoder()
data["Features"] = label_encoder.fit_transform(data["Features"])

# Separate features and target variable
X = data.drop(["Datetime", "Features", "Fall"], axis=1)
y = data["Fall"]

# Convert all columns in X to float32
X = X.astype('float32')  # Convert all data to float type

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition
model = Sequential()
model.add(LSTM(units=64, activation="relu", return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))  # Binary classification

# Model compilation
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Model training
model.fit(X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1), y_train, epochs=50, validation_data=(X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1), y_test))

# Evaluation on test set
y_pred = model.predict(X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1))
y_pred = (y_pred > 0.5).astype(int)  # Thresholding for class prediction

loss, accuracy = model.evaluate(X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1), y_test) # Reshape X_test for the LSTM layer
print("Test Accuracy:", accuracy)


recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Recall Score:", recall)
print("Precision Score:", precision)
print("F1 Score:", f1)

# Hyperparameter Tuning (Optional)
# Use techniques like GridSearchCV to find the best hyperparameter combination
# for your model (e.g., number of LSTM units, activation functions, learning rate)

Epoch 1/50
115/115 [==============================] - 11s 66ms/step - loss: 49197.9766 - accuracy: 0.6896 - val_loss: 36288.1602 - val_accuracy: 0.7005
Epoch 2/50
115/115 [==============================] - 4s 32ms/step - loss: 19944.1348 - accuracy: 0.7230 - val_loss: 10647.6973 - val_accuracy: 0.7432
Epoch 3/50
115/115 [==============================] - 4s 32ms/step - loss: 2575.6074 - accuracy: 0.6604 - val_loss: 35394.2227 - val_accuracy: 0.6415
Epoch 4/50
115/115 [==============================] - 5s 42ms/step - loss: 35852.3008 - accuracy: 0.6620 - val_loss: 98431.2969 - val_accuracy: 0.6842
Epoch 5/50
115/115 [==============================] - 5s 40ms/step - loss: 11267.1592 - accuracy: 0.6645 - val_loss: 755.9734 - val_accuracy: 0.6667
Epoch 6/50
115/115 [==============================] - 4s 32ms/step - loss: 194713.3438 - accuracy: 0.7003 - val_loss: 187396.8750 - val_accuracy: 0.7290
Epoch 7/50
115/115 [==============================] - 4s 33ms/step - loss: 11921.8926 - accura

In [14]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score

# Load data from CSV
# data = pd.read_csv("fall_data.csv")

# Define a function to map features to fall/no-fall categories
def map_features(features):
  # Assuming 'features' is now an integer representing the encoded feature
  fall_activities = [0, 1, 2]  # Replace with the encoded values for fall activities
  if features in fall_activities:
    return 0  # Fall
  else:
    return 1  # No Fall

# Apply mapping function to 'Features' column and create a new 'Fall' column
data["Fall"] = data["Features"].apply(map_features)

# Encode categorical columns (optional, if needed)
label_encoder = LabelEncoder()
categorical_cols = ["Datetime"]  # Assuming only Datetime is categorical
# Reshape the output of fit_transform to be a 2D array
data[categorical_cols] = label_encoder.fit_transform(data[categorical_cols]).reshape(-1, 1)

# Separate features and target variable
X = data.drop(["Fall"], axis=1)
y = data["Fall"]

# Convert all columns in X to float32
X = X.astype('float32')  # Convert all data to float type

# Reshape features for CNN (add channel dimension)
X = X.values.reshape(X.shape[0], X.shape[1], 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition
model = Sequential()

# CNN layer to extract features
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))  # Reduce dimensionality
model.add(Dropout(0.2))  # Dropout for regularization

# Remove the Flatten layer
# model.add(Flatten())

# LSTM layers for sequential processing
model.add(LSTM(units=64, activation="relu", return_sequences=True)) # The output of this LSTM layer will be 3D
model.add(Dropout(0.2))  # Dropout for regularization
model.add(LSTM(units=32, activation="relu")) # This LSTM layer now receives a 3D input

# Output layer for fall prediction
model.add(Dense(1, activation="sigmoid"))

# Model compilation
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Model training
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluation on test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Thresholding for class prediction

loss, accuracy = model.evaluate(X_test)
print("Test Accuracy:", accuracy)


recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Recall Score:", recall)
print("Precision Score:", precision)
print("F1 Score:", f1)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/50
115/115 [==============================] - 10s 28ms/step - loss: 33644.5391 - accuracy: 0.6896 - val_loss: 7446.3584 - val_accuracy: 0.7650
Epoch 2/50
115/115 [==============================] - 3s 24ms/step - loss: 27549.7832 - accuracy: 0.6984 - val_loss: 30203.5449 - val_accuracy: 0.7760
Epoch 3/50
115/115 [==============================] - 3s 23ms/step - loss: 39842.7539 - accuracy: 0.6560 - val_loss: 11561.0977 - val_accuracy: 0.6186
Epoch 4/50
115/115 [==============================] - 4s 32ms/step - loss: 45737.9805 - accuracy: 0.6913 - val_loss: 10524.9062 - val_accuracy: 0.7683
Epoch 5/50
115/115 [==============================] - 4s 34ms/step - loss: 24362.4453 - accuracy: 0.6888 - val_loss: 25631.5059 - val_accuracy: 0.6492
Epoch 6/50
115/115 [==============================] - 3s 23ms/step - loss: 30671.7637 - accuracy: 0.7104 - val_loss: 7582.4097 - val_accuracy: 0.7978
Epoch 7/50
115/115 [==============================] - 3s 23ms/step - loss: 23119.0234 - accurac